In [1]:
import time
import math
import numpy as np
import scipy.sparse as sp
import scipy.linalg as la
from scipy.spatial import distance_matrix 
from tqdm import tqdm

import dask.array as da
import dask_distance as dd
from dask import delayed
from dask.distributed import Client, progress


client = Client(processes=False, threads_per_worker=4,
                n_workers=3, memory_limit='2GB')
client


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://192.168.1.239:8787/status,
Dashboard: http://192.168.1.239:8787/status,Workers: 3
Total threads: 12,Total memory: 5.59 GiB
Status: running,Using processes: False
Comm: inproc://192.168.1.239/13596/1,Workers: 3
Dashboard: http://192.168.1.239:8787/status,Total threads: 12
Started: Just now,Total memory: 5.59 GiB
Comm: inproc://192.168.1.239/13596/5,Total threads: 4
Dashboard: http://192.168.1.239:53972/status,Memory: 1.86 GiB
Nanny: None,


In [3]:
a, b, c = 2, 2, 2
lat_res = 10

tot_atoms = (lat_res ** 3)

#relation = da.zeros((3*tot_atoms, 3*tot_atoms), chunks = (tot_atoms, tot_atoms))
#relation

In [4]:
points = np.array([[i * a, j * b, k * c] 
                   for k in range(lat_res) 
                   for j in range(lat_res) 
                   for i in range(lat_res)])

print(sys.getsizeof(points)/(1024*1024*1024))
points

1.12876296043396e-05


array([[ 0,  0,  0],
       [ 2,  0,  0],
       [ 4,  0,  0],
       ...,
       [14, 18, 18],
       [16, 18, 18],
       [18, 18, 18]])

In [5]:
euc = dd.cdist(points, points) + np.identity(tot_atoms)
euc
euc.compute()


array([[ 1.        ,  2.        ,  4.        , ..., 29.05167809,
        30.06659276, 31.17691454],
       [ 2.        ,  1.        ,  2.        , ..., 28.14249456,
        29.05167809, 30.06659276],
       [ 4.        ,  2.        ,  1.        , ..., 27.34958866,
        28.14249456, 29.05167809],
       ...,
       [29.05167809, 28.14249456, 27.34958866, ...,  1.        ,
         2.        ,  4.        ],
       [30.06659276, 29.05167809, 28.14249456, ...,  2.        ,
         1.        ,  2.        ],
       [31.17691454, 30.06659276, 29.05167809, ...,  4.        ,
         2.        ,  1.        ]])

In [6]:
#term1 = da.zeros((tot_atoms * 3, tot_atoms * 3), chunks = (tot_atoms, tot_atoms))
term1 = []

@delayed
def getterm1(points, x1, x2, y1, y2):
    
    p1 = ((points[x2][x1] - points[y2][x1]) *
          (points[x2][y1] - points[y2][y1]) *
          3)
    
    return p1

for i in tqdm(range(0, 3 * tot_atoms)):
    
        x1 = i % 3
        x2 = i // 3
    
        for j in range(0, 3 * tot_atoms):
        
            y1 = j % 3
            y2 = j // 3
            
            p2 = getterm1(points, x1, x2, y1, y2)
            term1.append(p2)
            
term1 = np.reshape(term1, (3 * tot_atoms, 3 * tot_atoms))

  8%|▊         | 232/3000 [00:27<05:17,  8.72it/s]distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 1.31 GiB -- Worker memory limit: 1.86 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 1.31 GiB -- Worker memory limit: 1.86 GiB
  8%|▊         | 238/3000 [00:28<04:35, 10.01it/s]distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 1.34 GiB -- Worker memory limit: 1.86 GiB
distributed.worker - WARNING - Unmanaged m

KeyboardInterrupt: 

distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 2.01 GiB -- Worker memory limit: 1.86 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 2.01 GiB -- Worker memory limit: 1.86 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 2.01 GiB -- Worker memory limit: 1.86 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see 

def term2_3(x, euc):
    
    # creation of the kronekar delta matrix
    x = np.arange(3 * x)
    kron = (abs(np.meshgrid(x, x)[0] - np.meshgrid(x, x)[1]) % 3 == 0)
    kron = kron.astype(int) - np.identity(len(kron))

    # repearing euclidian matrix 3 times in both dimensions
    # any way of doing this without storing it in memory, ie just pointing?
    # look into np.where for this
    new_euc = np.repeat(np.repeat(euc, [3], axis = 0), 3, axis = 1)
        
    # matrix-wise calculation of term 2 and term 3
    term2 = np.power(np.multiply(new_euc, kron), 2)
    term3 = np.power(new_euc, 5)
    
    return term2, term3

term1 = term1(tot_atoms, points)
term2, term3 = term2_3(tot_atoms, euc)

relation = np.divide((term1 - term2), term3)

end = time.perf_counter()
print(end - start)

relation_eig = la.eigvalsh(relation)
extreme_eig = [min(relation_eig), max(relation_eig)]
extreme_a = [1/extreme_eig[0], 1/extreme_eig[1]]
fig_o_merit = extreme_a[0]